In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker build -t tensorflow-training:2.12.0-cpu-py310-netflix -f Dockerfile.train .

In [ ]:
!docker images

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='tensorflow-training:2.12.0-cpu-py310-netflix',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/netflix_titles.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!tar -xzf ./data/output/model.tar.gz

In [ ]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-netflix"
echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 737368952498.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker push 737368952498.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-netflix

In [ ]:
!docker tag tensorflow-training:2.12.0-cpu-py310-netflix 737368952498.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-netflix